In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


import getopt, sys
import pandas as pd
import numpy as np
import qiime2
import time

from qiime2 import Metadata

Created a match_controls_forPlot.py file to make the plots below. It makes changes to file data_plot.csv created below. match_controls_forPlot.py is the same as match_controls.py except for the ability to change data_plot.csv. The only time that should be affected is load since the dataset and conditions are new things that are stored at that phase.

The file names for match, null, keep, case, and control indicate the conditions. Files with no conditions are the base line. r means range of exeptable samples. L, M, S usually mean Large, Medium, and Small respectivly except when they follow a number. In the case of L a number in front of it indicate how many lines the file is.

# notes

python match_controls.py --verbose --inputdata sk_files/sk_hc_merged.txt --nullvalues sk_files/skit_nullvalues.txt --case sk_files/skit_case.txt --control sk_files/skit_control.txt --match sk_files/skit_match.txt --output outputFiles/sk_output_testKeep.csv 


python match_controls.py --inputdata emp_files/emp_qiime_mapping_release1.tsv --output outputFiles/emp_output.csv --keep emp_files/emp_keep.txt  --control emp_files/emp_control.txt --case  emp_files/emp_case.txt --match emp_files/emp_match.txt --verbose


python match_controls.py --inputdata psych_files/psych_bins_saliva.txt --output outputFiles/psych_output_all.csv --keep psych_files/psych_bins_saliva_filter.txt --control psych_files/psych_bins_saliva_control.txt --case psych_files/psych_bins_saliva_case.txt --match psych_files/psych_bins_saliva_match.txt --nullvalues psych_files/psych_bins_saliva_null_values.txt --verbose


unknown if     index_header_name = afterExclusion_MD.id_header
is still usefull for merging

must be in folder with input data to use it in load 

# stable marriage debugging
not all samples are matched properly if two cases have same amount of matches and they overlap.

In [ ]:

def order_keys(dictionary):
    '''
    orders the keys of a dictionary so that they can be used properly as the freemen of stable marriage. In order greatest to least since pop is used to get least freeman and pop takes the right most entry. 

    Parameters
    dictionary
        dictionary of cases or controls with their matching controls or cases ordered 
        by rising abundance
    
    Return
    keys_greatest_to_least: list
        contains keys in order of greatest to least amount of samples they match to
    '''
    keys_greatest_to_least = []
    for key in dictionary:
        if keys_greatest_to_least == [] and len(dictionary[key])!=0:
            keys_greatest_to_least.append(key)
            continue
        abundance_of_key_values = len(dictionary[key])
        index = 0
        for list_key in keys_greatest_to_least:
            abundance_of_list_key_values = len(dictionary[list_key])
            
            if abundance_of_key_values > abundance_of_list_key_values:
                keys_greatest_to_least.insert(index, key)
                break
            if abundance_of_key_values == abundance_of_list_key_values:
                    if list_key >= key:
                        keys_greatest_to_least.insert(index, key)
                        break
            index = index + 1
        if index == len(keys_greatest_to_least):
            keys_greatest_to_least.append(key)

    return keys_greatest_to_least

def stable_marriage(case_dictionary, pref_counts_cont, pref_counts_case):
    #first make master copy
    master_copy_of_case_dict = case_dictionary.copy()
    #cut out keys in case_dictionary that have no possible matches
    list_of_keys = []
    for key in case_dictionary:
        list_of_keys.append(key)
    for key in list_of_keys:
        if len(case_dictionary[key])==0:
            case_dictionary.pop(key,None)
    
    free_keys = order_keys(case_dictionary)
    for key in free_keys:
        if pref_counts_case[key]==0:
            case_dictionary.pop(key,None)
            
            
    print(free_keys)
    one_to_one_match_dictionary = {}
    while free_keys :
        key = free_keys.pop()
        #print(key)
        print(case_dictionary)
        if case_dictionary[key]==[]:
            continue
        #get the highest ranked woman that has not yet been proposed to
        entry = case_dictionary[key].pop()
        #print(case_dictionary)
                
        if entry not in one_to_one_match_dictionary: 
            one_to_one_match_dictionary[entry] = key
            #remove key to reorder but this my not be the best if a switch is needed later
            if case_dictionary[key]==[]:
                case_dictionary.pop(key,None)
            for case_key in case_dictionary:
                if entry in case_dictionary[case_key]:
                    case_dictionary[case_key].remove(entry)
                    if case_dictionary[case_key] == []:
                        case_dictionary[case_key] = []
            #reorder keys
            free_keys = order_keys(case_dictionary)
            print("match %s to %s"%(entry,key))
            
        else:
            print("switch needed for key %s and entry %s"%(key,entry))
            key_in_use = one_to_one_match_dictionary [entry]
            if pref_counts_case[key] < pref_counts_case[key_in_use]:
                one_to_one_match_dictionary[entry] = key
                free_keys.append(key_in_use)
            else:
                free_keys.append(key)
                
    return one_to_one_match_dictionary

def test_stable_marriage():
    case_dictionary = {'14': ['15', '17'], '25': [], '19': ['20'], '21':[], '6': [], '9': ['10'], '3': [], '7': ['8'], 
                       '18': ['17'], '23': [], '16': ['13', '15'], '27': [], '11': ['12']}
    control_match_count_dictionary = {'10': 1, '15': 2, '17': 2, '12': 1, '8': 1, '20': 1, '13': 1}
    case_match_count_dictionary = {'23': 0, '6': 0, '14': 2, '21': 0, '16': 2, '11': 1, '19': 1, '9': 1, '27': 0, '7': 1,
                                   '3': 0, '25': 0, '18': 1}

    case_to_control_match = stable_marriage(case_dictionary, control_match_count_dictionary, case_match_count_dictionary)
    test_output  = {'15': '16', '12': '11', '8': '7', '10': '9', '17': '18', '20': '19'}
    if case_to_control_match != test_output:
        print("stable marriage fails. \nOutput should be {'15': '16', '12': '11', '8': '7', '10': '9', '17': '18', '20': '19'} \nOutput was")
        print(case_to_control_match)
        
        return False
    return True
print(test_stable_marriage())

In [ ]:
test_unorderedDict = {'key_1':['a','d','b','e','c'], 'key_2':['a','f','g',],  'key_3':['f','e','e','c','a'], 
                      'key_4':['a', 'd']}
test_frequencies = {'a':4,'b':1,'c':2,'d':2,'e':3, 'f':2 , 'g':1}
for k in test_unorderedDict:
    test_unorderedDict[k] = sorted(test_unorderedDict[k])
    test_unorderedDict[k] = sorted(test_unorderedDict[k], key = lambda x:test_frequencies[x] ) 
#, key=lambda x: len (test_unorderedDict[x])
  
test_unorderedDict

In [ ]:
test_unorderedDictLen={'2':['a','b'], '1':['a'],  '3':['a','b','c'],  '5':['a','b','c','d','e'], '4':['a','b','c','d']}

order = sorted(test_unorderedDictLen, key=lambda x: len(test_unorderedDictLen[x]), reverse=True)
  
order

old entry ordering


In [ ]:
dictionary_to_return = dictionary.copy()
    for key in dictionary_to_return:
        unordered_array = dictionary_to_return[key]
        if len(unordered_array) == 0:
            continue
        #ordered_array will store the elements linked to key in their proper order
        ordered_array = []
        count = 0
        while count < len(unordered_array):
            value = unordered_array[count]
            index = 0 #this index is for going through when sorting
            while index < len(ordered_array):
                ordered_value = ordered_array[index]
                if value_frequency[ordered_value] > value_frequency[value]:
                    ordered_array.insert(index,value)
                    break
                if value_frequency[ordered_value] == value_frequency[value]:
                    if ordered_value >= value:
                        ordered_array.insert(index,value)
                        break
                index = index + 1
            if index == len(ordered_array):
                ordered_array.insert(index,value)

            count = count + 1
        #overrides the unordered array linked to key with its ordered array
        dictionary_to_return[key] = ordered_array

old sorting

In [ ]:
for key in dictionary:
        if keys_greatest_to_least == []:
            keys_greatest_to_least.append(key)
            continue
        abundance_of_key_values = len(dictionary[key])
        index = 0
        for list_key in keys_greatest_to_least:
            abundance_of_list_key_values = len(dictionary[list_key])
            if abundance_of_key_values > abundance_of_list_key_values:
                keys_greatest_to_least.insert(index, key)
                break
            if abundance_of_key_values == abundance_of_list_key_values:
                    if list_key >= key:
                        keys_greatest_to_least.insert(index, key)
                        break
            index = index + 1
        if index == len(keys_greatest_to_least):
            keys_greatest_to_least.append(key)

In [ ]:
csvdata_match = pd.read_csv('./unitTest_files/unit_output.csv', sep = '\t' )
csvdata_match['id'] = csvdata_match['id'].astype('object')
#print( csvdata_match['id'] )
csvdata_match = csvdata_match.set_index('id').astype('object')
csvdata_match.index = csvdata_match.index.map(str)

In [ ]:
def merge_case_controlDF_and_afterExclutionMD(afterExclusion_MD, case_controlDF):
    '''
    Combines case_controlDF with afterExclution_MD and returns it as a metadata object
    
    Parameters
    ----------
    afterExclusion_MD : Metadata object
        Metadata object with unwanted samples filtered out
        
    case_controlDF : dataframe
        dataframe with one column named case_control. The indexs are the same as the indexs of afterExclution_MD  
        values reflect if the index is a case, control, or Undefined
        
    Returns
    -------
    Metadata(returnedMD) : Metadata object
        Metadata object with unwanted samples filtered out and a case_control column that reflects if the index is 
        a case, control, or Undefined    
    '''
    #turns case_controlDF into a metadata object
    case_controlMD = Metadata( case_controlDF)
    
    index_header_name = afterExclusion_MD.id_header
    #merges afterExclution_MD and case_controlMD into one new metadata object
    mergedMD = Metadata.merge(afterExclusion_MD, case_controlMD)

    return mergedMD

In [ ]:
plotFrame = pd.DataFrame(columns=['dataset','conditions','load','keep','case_control','null_filter','match','total'])
plotFrame.index.name = 'id'
plotFrame.to_csv('data_plot.csv', sep = '\t')
#plotFrame = plotFrame.append(pd.Series( {'dataset':1, 'conditions':1, 'case_control':(1- 2), 'match':(2- 1), 'total':(2-1), 'null_filter':(2 - 1), 'keep':(2 - 1), 'load':(2 - 1)}).rename('id_value') )
plotFrame

In [ ]:
'''
this one has 2 lines of conditions for --keep
'''
%run match_controls_forPlot.py --id ag_base --conditions none -i ag_map_with_alpha.txt --output agp_output.csv -k agp_keep.txt --control agp_control.txt --case=agp_case.txt --nullValues agp_nulls.txt --match agp_match.txt 


In [ ]:
'''
this one has 1 line of conditions for --keep
'''
%run match_controls_forPlot.py --id ag_1 --conditions keep_1L -i ag_map_with_alpha.txt --output agp_output1.csv -k agp_keep_1L.txt --control agp_control.txt --case=agp_case.txt --nullValues agp_nulls.txt --match agp_match.txt 


In [ ]:
'''
base conditions

'''

%run match_controls_forPlot.py  --id sk_base --conditions none -i sk_hc_merged.txt --nullValues agp_nulls.txt --case skit_case.txt --control skit_control.txt --match skit_match.txt --output sk_output_20.csv 



In [ ]:
'''
this one has 1 line of conditions for --control and --case

this one has a medium range in conditions for --keep
--match also has a medium range
'''

%run match_controls_forPlot.py --id ag_2 --conditions keep_rL -i ag_map_with_alpha.txt --output agp_output2.csv -k agp_keep_rL.txt --control agp_control_1L.txt --case=agp_case_1L.txt --nullValues agp_nulls.txt --match agp_match_rM.txt 


In [ ]:
'''
this one has is the base for emp
'''
%run match_controls_forPlot.py --id emp_base --conditions none -i emp_qiime_mapping_release1.tsv --output emp_output.csv --keep emp_keep.txt  --control emp_control.txt --case  emp_case.txt --match emp_match.txt
%tb

In [ ]:
plotDF = pd.read_csv('data_plot.csv',  sep = '\t').set_index('id')
plotDF

In [ ]:
'''
plots all the different times to complete each step 
total time is not included in the plot since it makes the the resolution between other points in the plot
'''
index_to_symbol = {'ag_base':'b', 'ag_1':'r', 'ag_2':'y', 'emp_base':'g', 'emp_1':'c'
                  }

for index, row in plotDF.iterrows():
    plt.plot(('loud', 'keep', 'case_control', 'null_filter', 'match'), (row['load'], row['keep'], row['case_control'], 
       row['null_filter'], row['match']), index_to_symbol[index], label = index)
    
    #plt.plot('total', row['total'], index_to_symbol[index])

plt.legend()
plt.ylabel('Time (sec)')    
plt.xlabel('Stage')
plt.show()

dataframe = pd.read_csv('emp_qiime_mapping_release1.tsv', sep='\t')
dataframe
Metadata.load( 'emp_qiime_mapping_release1.tsv' ).to_dataframe()['qc_filtered']

In [ ]:

#from before click implemented

    if null_values_lines_input == False or match_condition_lines_input == False:
        prepped_for_matchMD = case_controlMD
    else:
        prepped_for_matchMD= filter_prep_for_matchMD(case_controlMD, match_condition_lines_input, null_values_lines_input)

    tprepped = time.clock()
    print('time to prep Metadata information for match is %s'%(tprepped - tkeep))

    if match_condition_lines_input != False:
        matchedMD = match_samples( prepped_for_matchMD, match_condition_lines_input )
        matchedMD.to_dataframe().to_csv(outputFileName, sep = '\t')

    tmatch = time.clock()
    tend = time.clock()
    print('time to match is %s'%(tmatch- tprepped))
    print('time to do everything %s'%(tend-tstart))

    

#look in to flag for exclude_query_lines_input
if exclude_query_lines_input != False:
    afterExclusionMD = keep_samples(originalMD, exclude_query_lines_input)
else:
    afterExclusionMD = originalMD

tkeep = time.clock()
print('time to filter out unwanted samples is %s'%(tkeep - tloadedFiles))

if case_query_lines_input != False and control_query_lines_input != False:
    ids = afterExclusionMD.get_ids()
    case_control_Series = pd.Series( ['Unspecified'] * len(ids), ids)
    '''
    ['Unspecified'] * len(ids) creates a list of elements. The list is the
    same length as ids. All the elements are 'Unspecified'
    '''
    case_control_Series.index.name = afterExclusionMD.id_header
    case_controlDF = case_control_Series.to_frame('case_control')
    case_control_dict = {'case':case_query_lines_input, 'control':control_query_lines_input }

    case_controlMD = determine_cases_and_controls(afterExclusionMD, case_control_dict, case_controlDF)
else:
    afterExclusionMD.to_dataframe().to_csv(outputFileName, sep = '\t')
    print('keep exit')
    break



In [ ]:

correctOutFrame = open('./unitTest_files/%s'%('test_case.txt'),'r').readlines()


smallTestFile = pd.DataFrame(index = [1,2,3], data = { 'bmi': ['normal','obese','overweight'], 'age': [23,42,11] } )
largeTestFile = pd.read_csv('./unitTest_files/test_data.csv', sep = '\t' ).set_index('id')
truncatedlargeTestFile = pd.read_csv('./unitTest_files/test_data_2.csv', sep = '\t' ).set_index('id')


csvdata_keep = Metadata.load('./unitTest_files/unit_keep.csv').to_dataframe()
csvdata_case_control = Metadata.load('./unitTest_files/unit_control.csv').to_dataframe()
csvdata_filter = Metadata.load('./unitTest_files/unit_filtered.csv').to_dataframe()
csvdata_match = Metadata.load('./unitTest_files/unit_output.csv').to_dataframe()
#print(csvdata_match)


tstart = time.clock()


# reading in commandline arguments
all_arguments = sys.argv
# selecting all arguments after python file name
argumentList = all_arguments[1:]
unixOptions = "i:k:c:e:n:m:o:"
gnuOptions = ["inputData=", "keep=", "control=", "case=", "nullValues=", "match=", "output="]

try:
    arguments, values = getopt.getopt(argumentList, unixOptions, gnuOptions)
except getopt.error as err:
    # output error, and return with an error code
    print (str(err))
    sys.exit(2)


#metadata file
file_of_metadata = ''
user_input_file_name_exclude = ''
user_input_file_name_control = ''
user_input_file_name_experiment = ''
user_input_file_null_values = ''
user_input_file_name_match = ''

# evaluate given options
#print(arguments)

for currentArgument, currentValue in arguments:
    if currentArgument in ("-v", "--verbose"):
        print ("enabling verbose mode")
    elif currentArgument in ("-h", "--help"):
        print ("displaying help")
    elif currentArgument in ("-i", "--inputData"):
        file_of_metadata = currentValue
    elif currentArgument in ("-k", "--keep"):
        user_input_file_name_exclude = currentValue
    elif currentArgument in ("-c", "--control"):
        user_input_file_name_control = currentValue
    elif currentArgument in ("-e", "--case"):
        user_input_file_name_experiment = currentValue
    elif currentArgument in ("-n", "--nullValues"):
        user_input_file_null_values = currentValue
    elif currentArgument in ("-m", "--match"):
        user_input_file_name_match = currentValue
    elif currentArgument in ("-o", "--output"):
        outputFileName = currentValue

if file_of_metadata == '':
    print('metadata file not found')
    sys.exit(2)
if outputFileName == '':
    print('output put file name not entered')
    sys.exit(2)
#read metadata file into metadata object
originalMD = Metadata.load( file_of_metadata )



#each line is a sqlite query to determine what samples to keep
exclude_query_lines_input = match_controls.get_user_input_query_lines(user_input_file_name_exclude)
#each line is a sqlite query to determine what samples to label control
control_query_lines_input = match_controls.get_user_input_query_lines(user_input_file_name_control)
#each line is a sqlite query to determine what samples to label case
case_query_lines_input = match_controls.get_user_input_query_lines(user_input_file_name_experiment)
null_values_lines_input = match_controls.get_user_input_query_lines(user_input_file_null_values)

'''
each line is tab seperated
the first element is the type of match: range or exact
    range matches samples if the numerical values compared are with in some other number of eachother
        this is only to be used with numerical values
    exact matches samples if the values compared are exactly the same
        this can be used for strings and numbers
the second element is the column to compare values of for the case and control samples
the third element is the range number or = (if the match type is exact)
    this determines how far away a sample can be from another sample for the given column to be matched
'''
match_condition_lines_input = match_controls.get_user_input_query_lines(user_input_file_name_match)

tloadedFiles = time.clock()
print('time to load input files is %s'%(tloadedFiles - tstart))








if test_orderDict() == False:
    sys.exit(0)
if test_order_keys() == False:
    sys.exit(0)
if test_stable_marriage() == False:
    sys.exit(0)


if exclude_query_lines_input != False:
    afterExclusionMD = match_controls.keep_samples(originalMD, exclude_query_lines_input)
else:
    afterExclusionMD = originalMD

keep = test_keep_samples()
print("keep_samples function works correctly is %s"%(keep))

tkeep = time.clock()
print('time to filter out unwanted samples is %s'%(tkeep - tloadedFiles))

if case_query_lines_input != False and control_query_lines_input != False:
    ids = afterExclusionMD.get_ids()
    case_control_Series = pd.Series( ['Unspecified'] * len(ids), ids)
    '''
    ['Unspecified'] * len(ids) creates a list of elements. The list is the
    same length as ids. All the elements are 'Unspecified'
    '''
    case_control_Series.index.name = afterExclusionMD.id_header
    case_controlDF = case_control_Series.to_frame('case_control')
    case_control_dict = {'case':case_query_lines_input, 'control':control_query_lines_input }

    case_controlMD = match_controls.determine_cases_and_controls(afterExclusionMD, case_control_dict, case_controlDF)
else:
    afterExclusionMD.to_dataframe().to_csv(outputFileName, sep = '\t')
    print('keep exit')
    sys.exit(0)

cas_con = test_determine_cases_and_controls()
print("determine_cases_and_controls function works correctly is %s"%(cas_con))

if null_values_lines_input == False or match_condition_lines_input == False:
    prepped_for_matchMD = case_controlMD
else:
    prepped_for_matchMD= match_controls.filter_prep_for_matchMD(case_controlMD, match_condition_lines_input, null_values_lines_input)

filtered = test_filter_prep_for_matchMD()
print("filter_prep_for_matchMD function works correctly is %s"%(filtered))

tprepped = time.clock()
print('time to prep Metadata information for match is %s'%(tprepped - tkeep))

if match_condition_lines_input != False:
    matchedMD = match_controls.match_samples( prepped_for_matchMD, match_condition_lines_input )
    matchedMD.to_dataframe().to_csv(outputFileName, sep = '\t')

match = test_match_samples()
print("match_samples function works correctly is %s"%(match))

tmatch = time.clock()
tend = time.clock()
print('time to match is %s'%(tmatch- tprepped))
print('time to do everything %s'%(tend-tstart))